## 层次分析法
### Author：Henry   Time：2022/07/17

1. 方法概述
- 层次分析法是一种针对多目标、多准则问题的决策方法
- 特点：层次化、数量化

2. 主要应用场景
- 选择最佳方案
- 评价类
- 指标体系的优选

3. 步骤方法
    1. 建立层次结构模型
        - 最高层（top）：决策的目的，如：找工作
        - 中间层（middle）：决策的影响因素，如：考虑收入、假期、公司名声等等
        - 最底层（bottom）：决策的备选方案，如：可选公司
    2. 构造判断（成对比较）矩阵
        - 不把所有因素放在一起比较，而是两两比较
        - 通常，因素不应超过9个
        - 标度：
            |    标度   |       含义       |
            |:---------:|:----------------:|
            |       1   |       同等重要    |
            |       3   |       稍微重要    |
            |       5   |       明显重要    |
            |       7   |       强烈重要    |
            |       9   |       极端重要    |
            | 2，4，6，8|       介于之间    |
            |    倒数   |$a_{ij} = \frac{1}{a_{ji}}$|
        - 比较矩阵：
            - 若 $a_{ij} * a_{jk} = a_{ik}$，则称为一致阵（需了解一致阵的性质）
            - 否则称为非一致阵，不一致阵有限制范围 
    3. 层次单排序及其一致性检验
        - 层次单排序：用比较矩阵的最大特征值对应的特征向量(归一化)作为权向量
        - 一致性检验：
            - 当为一致阵时特征值=矩阵维数，即 $\lambda = n$
            - 当为非一致阵时 $\lambda = n$，且 $\lambda$ 越大，不一致程度越高
            - 一致性比率 $CR = \frac{CI}{RI}$，当 $CR < 0.1$ 时认为满足一致性
                - 一致性指标 $CI = \frac{\lambda - n}{n - 1}$
                - 随机一致性指标 RI 查表可得
                    | n | 1 |2 |3|4|5|6|7|8|9|10|11|
                    |:-:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
                    | RI| 0 |0 |0.58| 0.9| 1.12| 1.24| 1.32| 1.41| 1.45| 1.49| 1.51|
    4. 层次总排序及其一致性检验
        - 层次总排序：bottom先对middle求权重，再两组权重相乘求得对top的权重
        - 一致性检验：
            - $CR = \frac{m_{1}CI_{1} + …… + m_{n}CI_{n}}{m_{1}RI_{1} + …… + m_{n}RI_{n}}$ 


In [ ]:
import numpy as np

In [ ]:
class AnalyticHierarchyProcess:
    def __init__(self, middle, bottom):
        self.RI = np.array([0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49, 1.51])
        self.middle = middle
        self.bottom = bottom

    # 一致性检验
    def consistency_check(self, value_max, vector_max, n, mode="single"):
        if mode == "single":
            CI = (value_max - n) / (n - 1)
            RI = self.RI[n]
            CR = CI / RI
        elif mode == "all":
            CI = 0
            RI = 0
            for i in range(len(value_max)):
                CI += vector_max[i] * (value_max[i] - n) / (n - 1)
                RI += vector_max[i] * self.RI[n]
            CR = CI / RI
        if CR < 0.1:
            print(f"层次单排序一致性检验满足条件,CR={CR}")
        else:
            print(f"层次单排序一致性检验不满足条件,CR={CR}")

    # 获取比较矩阵最大特征值及其对应归一化的特征向量
    def get_value_vector_max(self, mat) -> tuple :
        value, vector = np.linalg.eig(self.middle)
        value_max = np.argmax(value)
        vector_max = (vector.T)[value_max]
        return (value_max, vector_max)

    def solution(self):

        # 层次单排序
        value_middle, vector_middle = self.get_value_vector_max(self.middle)
        self.consistency_check(value_middle, 0, self.middle.shape[0], "single")

        # 层次总排序
        value_bottom = [] 
        vector_bottom = [] 
        for i in self.bottom:
            value_tmp, vector_tmp = self.get_value_vector_max(i)
            value_bottom.append(value_tmp)
            vector_bottom.append(vector_tmp)     
        value_bottom = np.array(value_bottom)
        vector_bottom = np.array(vector_bottom)
        self.consistency_check(value_bottom, vector_middle, self.bottom.shape[0], "all")
        
        # 计算各个方案权重
        result = []
        for i in range(self.bottom.shape[0]):
            weight = 0
            for j in range(vector_bottom.shape[0]):
                weight += vector_bottom[j][i] * vector_middle[j]
            result.append(weight)

        return result


In [ ]:
middle = np.zeros((3,3))
bottom = np.zeros((3,3))
solution = AnalyticHierarchyProcess(middle, bottom) 
result = solution.solution()